In [4]:

# Required columns (DB schema)
required_columns = [
    "exchange",
    "exchange_token",
    "trading_symbol",
    "name",
    "instrument_type",
    "segment",
    "series",
    "isin",
    "expiry_date",
    "strike_price",
    "lot_size"
]

# Keep only existing columns
existing_columns = [c for c in required_columns if c in df.columns]

# Filter dataframe
final_df = df.loc[:, existing_columns].copy()
final_df    

NameError: name 'df' is not defined

In [22]:
import duckdb

# DB PATH
db_path = r"C:\Users\jallu\OneDrive\pgp\Python\Stock predictor\rubik-analytics\data\symbols\symbols.duckdb"

# CONNECT
con = duckdb.connect(db_path)

# CHECK TABLES
tables = con.execute("SHOW TABLES").fetchall()
print("Tables:", tables)

# ROW COUNT (change table name if needed)
row_count = con.execute("SELECT COUNT(*) FROM symbols").fetchone()[0]
print("Row count:", row_count)

# CLOSE
con.close()


Tables: [('schedulers',), ('series_lookup',), ('series_lookup_metadata',), ('symbols',), ('transformation_scripts',), ('upload_logs',)]
Row count: 176167


In [ ]:
import duckdb

# DB PATH
db_path = r"C:\Users\jallu\OneDrive\pgp\Python\Stock predictor\rubik-analytics\data\symbols\symbols.duckdb"

# CONNECT
con = duckdb.connect(db_path)

# GET COLUMN HEADERS
columns = con.execute("DESCRIBE symbols").fetchall()

print("Column headers:")
for col in columns:
    print(col[0])

# CLOSE
con.close()


Column headers:
id
exchange
trading_symbol
exchange_token
name
instrument_type
segment
series
isin
expiry_date
strike_price
lot_size
status
source
created_at
updated_at
last_updated_at


: 

In [3]:
for table in ["symbol_upload_logs", "symbols"]:
    count = con.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
    print(f"{table}: {count}")


ConnectionException: Connection Error: Connection already closed!

In [ ]:
import json
import os
import pandas as pd
from pathlib import Path
import duckdb
import sqlite3

# Path to connections data
base_path = Path(r"C:\Users\jallu\OneDrive\pgp\Python\Stock predictor\rubik-analytics")
connections_file = base_path / "data" / "connections" / "connections.json"
active_file = base_path / "data" / "connections" / "active_connection.json"

# Load connections
connections_data = {}
active_connections = {}

if connections_file.exists():
    with open(connections_file, 'r') as f:
        data = json.load(f)
        connections_data = {conn["id"]: conn for conn in data.get("connections", [])}

if active_file.exists():
    with open(active_file, 'r') as f:
        active_connections = json.load(f)

# Test connection status
def test_connection(conn_type, config):
    """Test if a database connection is working"""
    try:
        if conn_type == "sqlite" or conn_type == "sqlite3":
            path = config.get("path") or config.get("database")
            if path and os.path.exists(path):
                conn = sqlite3.connect(path)
                conn.execute("SELECT 1")
                conn.close()
                return "✓ Connected"
            return "✗ File not found"
        elif conn_type in ["duckdb", "duckdb_direct", "duckdb_sqlalchemy"]:
            path = config.get("path")
            if path:
                if os.path.exists(path):
                    con = duckdb.connect(path)
                    con.execute("SELECT 1")
                    con.close()
                    return "✓ Connected"
                return "✗ File not found"
            # Multi-database case
            return "✓ Configured"
        else:
            return "? Unknown type"
    except Exception as e:
        return f"✗ Error: {str(e)[:30]}"

# Display connections
print("=" * 100)
print("DATABASE CONNECTIONS STATUS")
print("=" * 100)
print()

if connections_data:
    # Create a list for display
    conn_list = []
    for conn_id, conn in connections_data.items():
        category = conn.get("category", "N/A")
        is_active = conn_id in active_connections.values()
        active_marker = "✓ ACTIVE" if is_active else "  "
        
        conn_type = conn.get("type", "N/A")
        name = conn.get("name", "N/A")
        
        # Get database path(s)
        config = conn.get("config", {})
        if "path" in config:
            db_path = config["path"]
            # Test connection
            conn_status = test_connection(conn_type, config)
        elif conn_type == "duckdb" and any(k in config for k in ["ohlcv", "indicators", "signals", "jobs"]):
            db_path = f"Multiple: {', '.join([k for k in config.keys() if k != 'path'])}"
            conn_status = "✓ Configured (multi-DB)"
        else:
            db_path = "N/A"
            conn_status = "? Not configured"
        
        conn_list.append({
            "Active": active_marker,
            "Name": name,
            "Type": conn_type,
            "Category": category,
            "Connection Status": conn_status,
            "Database Path": str(db_path)[:60] + "..." if len(str(db_path)) > 60 else str(db_path)
        })
    
    # Display as DataFrame
    df_connections = pd.DataFrame(conn_list)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', 70)
    print(df_connections.to_string(index=False))
    print()
    
    # Show active connections summary
    print("-" * 100)
    print("ACTIVE CONNECTIONS BY CATEGORY:")
    print("-" * 100)
    for category, conn_id in sorted(active_connections.items()):
        if conn_id:
            conn = connections_data.get(conn_id, {})
            config = conn.get("config", {})
            conn_type = conn.get("type", "N/A")
            status = test_connection(conn_type, config)
            print(f"  {category:20s} → {conn.get('name', conn_id):40s} ({conn_type:15s}) {status}")
        else:
            print(f"  {category:20s} → None")
    print()
    
    # Summary
    active_count = sum(1 for cid in connections_data.keys() if cid in active_connections.values())
    total_count = len(connections_data)
    print(f"Summary: {active_count} active out of {total_count} total connections")
else:
    print("No connections found.")

print()
print("=" * 100)


In [3]:
import yfinance as yf

df = yf.download(
    "RELIANCE.NS",
    start="2025-12-10",
    end="2025-12-12",
    interval="1d"
)

print(df.head())


C:\Users\jallu\AppData\Local\Temp\ipykernel_70964\2181961351.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(
[*********************100%***********************]  1 of 1 completed

Price             Close        High          Low         Open      Volume
Ticker      RELIANCE.NS RELIANCE.NS  RELIANCE.NS  RELIANCE.NS RELIANCE.NS
Date                                                                     
2025-12-10  1536.900024      1547.5  1531.400024  1534.000000     7991629
2025-12-11  1545.000000      1550.0  1524.000000  1536.900024     4706197


In [36]:
import sqlite3

# DB PATH
db_path = r"C:\Users\jallu\OneDrive\pgp\Python\Stock predictor\rubik-analytics\backend\data\auth\sqlite\auth.db"

# CONNECT
con = sqlite3.connect(db_path)
cur = con.cursor()

# CHECK TABLES
cur.execute("""
    SELECT name 
    FROM sqlite_master 
    WHERE type='table';
""")
tables = cur.fetchall()
print("Tables:", tables)

# ROW COUNT (change table name if needed)
table_name = "users"  # <-- replace with actual table name
cur.execute(f"SELECT COUNT(*) FROM {table_name}")
row_count = cur.fetchone()[0]
print("Row count:", row_count)

# CLOSE
con.close()


Tables: [('users',), ('access_requests',), ('feedback',), ('sessions',), ('feature_requests',), ('audit_logs',), ('sqlite_sequence',), ('symbols',), ('transformation_scripts',), ('symbol_upload_logs',), ('scheduled_ingestions',), ('ingestion_sources',), ('connections',)]
Row count: 3


In [3]:
import requests
import json
import pandas as pd
from datetime import datetime
import websocket
import threading


In [1]:

import requests
AUTH_URL = "https://auth.truedata.in/token"

USERNAME = "trial134"
PASSWORD = "sandy134"


url = "https://auth.truedata.in/token"

payload = {
    "username": USERNAME,
    "password": PASSWORD,
    "grant_type": "password"
}

resp = requests.post(url, data=payload, timeout=10)

print("Status:", resp.status_code)
print(resp.json())



Status: 200
{'access_token': '_0NWeomfYZ4_WtSSwQ0Sbl8_HwnjDXF5qfbqCTixV-Td1HsS_UTziRQLjmlY5DYpQOoT-RqoH9KAuIoJO54QE0IMg_YEME2z8NYbTuhysnCeyohVQTkZwkDD9I5gPOFa3C_3eNZHN5B1n07-BpYhphrYH_7pYiqNYMQrGCkudZE8augz61GtZBNtU0JwbRbK3WE4q5XXFU8wvTJKxcq3dOHo-mpYXKwO4AyYE2ZOWm-3DL_tqFE0WFkSMgMWrWZTSEasHzml1Upv_vVxvYsIPQ', 'token_type': 'bearer', 'expires_in': 56864, 'userName': 'trial134', '.expires': 'Mon, 05 Jan 2026 22:30:00 GMT', '.issued': 'Mon, 05 Jan 2026 06:42:15 GMT'}


In [2]:
token = resp.json()["access_token"]

headers = {
    "Authorization": f"Bearer {token}"
}

url = "https://corporate.truedata.in/getdescriptors?response=csv"

r = requests.get(url, headers=headers, timeout=10)

print("Status:", r.status_code)
print(r.text[:500])


Status: 200
id,descriptor,category
1,General,General Announcement/ General Updates
2,Acquisition,Announcement under Regulation 30 (LODR)
3,AGM,Shareholder Meeting / Postal Ballot
5,Allotment,Announcement under Regulation 30 (LODR)
6,Change_in_RTA,Announcement under Regulation 30 (LODR)
7,Amalgamation_Merger_Demerger,Corporate Action
8,Award_of_Order_Receipt_of_Order,Announcement under Regulation 30 (LODR)
9,Board_Meeting,Board Meeting
10,Bonus,Corporate Action
11,Book_Closure,Corporate Action
12,


In [ ]:
url = (
    "https://corporate.truedata.in/"
    "getannouncementsforcompaniesrange"
    "?response=json&from=250815 00:00:00&to=250817 23:59:59"
)

r = requests.get(url, headers=headers, timeout=15)

print("Status:", r.status_code)
print(r.text[:500])


Status: 200
{"status":"Success","Records":[["4519411","17-08-2025 23:27:06",532879,"","SSLEL_BSE","Sir Shadi Lal Enterprises Ltd","N","Announcement under Regulation 30 (LODR)-Newspaper Publication","Announcement under Regulation 30 (LODR)-Newspaper Publication","Notice of 91st Annual General Meeting & E-voting information published in the newspaper","Newspaper Publication","General_Announcements","",0,234.4,0],["4519410","17-08-2025 22:44:50",517170,"","EDVENSWA_BSE","Edvenswa Enterprises Ltd","N","Newspape


: 

In [19]:
url = "https://corporate.truedata.in/getSymbolClassification?symbol=ACC&response=json"

r = requests.get(url, headers=headers, timeout=10)

print("Status:", r.status_code)
print(r.json())


Status: 200
{'status': 'Success', 'Records': [{'symbol': 'ACC', 'company': 'ACC LIMITED', 'isin': 'INE012A01025', 'sector': 'Commodities', 'industry': 'Construction Materials', 'group': 'Cement & Cement Products', 'subgroup': 'Cement & Cement Products'}]}


In [20]:
ANNOUNCEMENT_ID = 706710  # example

url = f"https://corporate.truedata.in/announcementfile?id={ANNOUNCEMENT_ID}"

r = requests.get(url, headers=headers, timeout=15)

print("Status:", r.status_code)
print("Content-Type:", r.headers.get("Content-Type"))
print("Content-Length:", len(r.content))


Status: 200
Content-Type: application/pdf
Content-Length: 4947668


In [21]:
import websocket
import json

def on_message(ws, message):
    data = json.loads(message)
    print("\n📢 LIVE ANNOUNCEMENT")
    print("Company:", data.get("CompanyName"))
    print("Symbol NSE:", data.get("Symbol_Nse"))
    print("Headline:", data.get("HeadLine"))
    print("Time:", data.get("TradeDate"))

def on_open(ws):
    print("✅ WebSocket Connected")

def on_error(ws, error):
    print("❌ WebSocket Error:", error)

def on_close(ws):
    print("🔌 WebSocket Closed")

ws_url = f"wss://corp.truedata.in:9092?user={USERNAME}&password={PASSWORD}"

ws = websocket.WebSocketApp(
    ws_url,
    on_open=on_open,
    on_message=on_message,
    on_error=on_error,
    on_close=on_close
)

ws.run_forever()


✅ WebSocket Connected

📢 LIVE ANNOUNCEMENT
Company: None
Symbol NSE: None
Headline: None
Time: None

📢 LIVE ANNOUNCEMENT
Company: None
Symbol NSE: None
Headline: None
Time: None

📢 LIVE ANNOUNCEMENT
Company: None
Symbol NSE: None
Headline: None
Time: None

📢 LIVE ANNOUNCEMENT
Company: None
Symbol NSE: None
Headline: None
Time: None
❌ WebSocket Error: 
❌ WebSocket Error: on_close() takes 1 positional argument but 3 were given


True

In [8]:
import requests

URL = "https://api.truedata.in/getAllSymbols"

USERNAME = "trial134"
PASSWORD = "sandy134"

PARAMS = {
    "segment": "eq",
    "user": USERNAME,
    "password": PASSWORD
}

response = requests.get(URL, params=PARAMS, timeout=120)

print("HTTP Status:", response.status_code)

if response.status_code != 200:
    raise Exception("Failed to fetch symbols")

data = response.json()

print("Top-level keys:", data.keys())


HTTP Status: 200
Top-level keys: dict_keys(['status', 'Records'])


In [9]:
records = data.get("records") or data.get("Records")

if not records:
    raise Exception("No symbol records found")

print("Total symbols:", len(records))


Total symbols: 11691


In [10]:
import pandas as pd

df = pd.DataFrame(records)

print("Columns:")
print(df.columns.tolist())

df.head()


Columns:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,100000001,20MICRONS,EQ,INE144J01027,NSE,1,0.0,1970-01-01T00:00:00,20MICRONS,20MICRONS,...,2008-10-06T00:00:00,1,0,0.0,169.67-254.49,0,None,None,0,False
1,100000002,21STCENMGM,EQ,INE253B01015,NSE,1,0.0,1970-01-01T00:00:00,21STCENMGM,21STCENMGM,...,1995-05-03T00:00:00,1,0,0.0,48.05-50.01,0,None,None,0,False
2,100000004,3MINDIA,EQ,INE470A01017,NSE,1,0.0,1970-01-01T00:00:00,3MINDIA,3MINDIA,...,2004-08-13T00:00:00,500,0,0.0,28800-43200,0,None,None,0,False
3,100000005,3PLAND,EQ,INE105C01023,NSE,1,0.0,1970-01-01T00:00:00,3PLAND,3PLAND,...,1995-07-19T00:00:00,1,0,0.0,29.92-44.86,0,None,None,0,False
4,100000006,5PAISA,EQ,INE618L01018,NSE,1,0.0,1970-01-01T00:00:00,5PAISA,5PAISA,...,2017-11-16T00:00:00,5,0,0.0,300.65-450.95,0,None,None,0,False


In [11]:
OUTPUT_FILE = "truedata_all_symbols_eq.csv"

df.to_csv(OUTPUT_FILE, index=False)

print(f"✅ Saved {len(df)} symbols to {OUTPUT_FILE}")


✅ Saved 11691 symbols to truedata_all_symbols_eq.csv


In [ ]:
import requests

URL = "https://api.truedata.in/getAllSymbols"

PARAMS = {
    "segment": "eq",
    "user": "trial134",
    "password": "sandy134"
}

response = requests.get(URL, params=PARAMS, timeout=120)
data = response.json()

print("Top-level keys:", data.keys())   
data

Top-level keys: dict_keys(['status', 'Records'])


{'status': 'Success',
 'Records': [[100000001,
   '20MICRONS',
   'EQ',
   'INE144J01027',
   'NSE',
   1,
   0.0,
   '1970-01-01T00:00:00',
   '20MICRONS',
   '20MICRONS',
   16921,
   None,
   '20 MICRONS LTD',
   '2008-10-06T00:00:00',
   1,
   0,
   0.0,
   '169.67-254.49',
   0,
   None,
   None,
   0,
   False],
  [100000002,
   '21STCENMGM',
   'EQ',
   'INE253B01015',
   'NSE',
   1,
   0.0,
   '1970-01-01T00:00:00',
   '21STCENMGM',
   '21STCENMGM',
   4,
   None,
   '21ST CENTURY MGMT SERVICE',
   '1995-05-03T00:00:00',
   1,
   0,
   0.0,
   '48.05-50.01',
   0,
   None,
   None,
   0,
   False],
  [100000004,
   '3MINDIA',
   'EQ',
   'INE470A01017',
   'NSE',
   1,
   0.0,
   '1970-01-01T00:00:00',
   '3MINDIA',
   '3MINDIA',
   474,
   None,
   '3M INDIA LIMITED',
   '2004-08-13T00:00:00',
   500,
   0,
   0.0,
   '28800-43200',
   0,
   None,
   None,
   0,
   False],
  [100000005,
   '3PLAND',
   'EQ',
   'INE105C01023',
   'NSE',
   1,
   0.0,
   '1970-01-01T00:00:00',

: 